# Car price model for all car companies 

Importing Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
files = ['audi','bmw','ford','hyundi','merc','skoda','toyota','vauxhall','vw']  # file names

Reading data from multiple files.

In [ ]:
dataframes = {}
for f in files:

  dataframes[f+'_data'] = pd.read_csv('../input/used-car-dataset-ford-and-mercedes/'+f+'.csv')
  dataframes[f+'_data']['company'] = [f]*dataframes[f+'_data'].shape[0]
  dataframes[f+'_data'] = dataframes[f+'_data'].reindex(columns = sorted(dataframes[f+'_data'].columns))
  print( f+'_data :' , dataframes[f+'_data'].info() )

In [ ]:
dataframes['hyundi_data'].rename(columns={"tax(£)": "tax"} , inplace = True)

In [ ]:
dataframes['hyundi_data'].info()

Creating a dataframe by concatinating data from allthe files.

In [ ]:
Carsdata = pd.DataFrame()
Carsdata = pd.concat(dataframes.values() )

In [ ]:
Carsdata.sample(10)

In [ ]:
Carsdata.info()

In [ ]:
Carsdata.year.loc[17726]= 2006

### Creating Pandas Profile report of dataframe.

In [ ]:
from pandas_profiling import ProfileReport

In [ ]:
profile = ProfileReport(Carsdata , title ='Pandas Profiling')  # report for data with outliers.

In [ ]:
# code snippit for removing outliers.
df = Carsdata
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print('Q1 :', Q1)
print('Q3 :', Q3)
print('IQR ', IQR)
dff = df[~( (df < (Q1 - 1.5*IQR))|(df > (Q3 + 1.5*IQR)) ).any(axis=1)]
dff.shape

In [ ]:
profile2 = ProfileReport(dff , title ='Pandas Profiling Report of Cars dataset')   # report for data without outliers.

In [ ]:
profile2

### Vizualization

In [ ]:
fig ,  ax = plt.subplots(1,2,figsize=(15, 5) )
ax[0].scatter( Carsdata.price ,  Carsdata.mileage)
ax[1].scatter( dff.price ,  dff.mileage)

plt.title(' Without Outliers Price VS Mileage ')
plt.xlabel('price')
plt.ylabel('Mileage')

* from the above graph we get to know that there is a negative corelation between the price and the mileage.

In [ ]:
Carsdata.boxplot(['mileage' ],by= ['engineSize'] , figsize=(18 ,5))

* from the we get to know that cars with engineSize 1.9 , 2.7 , 3.7 are very good mileage.

In [ ]:
Carsdata.boxplot(['price' ],by= ['engineSize'] , figsize=(18 ,5))

* from above two boxplots we get know that 1.9 engineSize is cost efficent as well as has long on road life.
* and also engine sizes 4.0 and 5.2 are the most costly ones. 

In [ ]:
Carsdata[Carsdata.engineSize == 1.9]

* from above we get the reason of engineSize 1.9 to be cheaper that is because it is an old modle when cars use to be cheaper as compaerd to now.
* The another reason of long road life is the fuel type that is 'Diesel', and transmission is 'Manual' , so it needs less fule . 

In [ ]:
Carsdata[Carsdata.engineSize == 4.0].fuelType.value_counts()

In [ ]:
Carsdata[Carsdata.engineSize == 4.0].transmission.value_counts()

* the reason of being so expensive of this engineSize 4.0 is the transmission i.e 'Semi-Auto' and 'Autometic'.

In [ ]:
Carsdata[Carsdata.engineSize == 5.2]

* the reason of being so expensive of this engineSize 5.2 is the transmission i.e 'Semi-Auto' and 'Autometic'.

In [ ]:
Carsdata.boxplot(['price','mileage'] , by = ['transmission'] , figsize=(15 , 4) )
dff.boxplot( ['price','mileage'] , by = ['transmission'] , figsize=(15 , 4) )

* Manual cars are cheaper and give good mileage were as Automatic and Semi-auto once are expensive and give less mileage.

In [ ]:
Carsdata.boxplot(['price','mileage'] , by= ['fuelType'] , figsize=(15 , 4) )
dff.boxplot(['price','mileage'], by= ['fuelType'] ,  figsize=(15 , 4) )

* cars with Hybrid & Diesel fueltype are expensive.
* cars with fueltype Petrol are less expensive.
* because of the fuel prices
  * cars with Diesel fueltype have more mileage
  * cars with Petrol fueltype have less mileage

In [ ]:
Carsdata.boxplot(['price']  , by= ['company'], figsize=(15 , 4) )
dff.boxplot(['price']  , by= ['company'], figsize=(15 , 4) )

* from the above graph we gets to know that 'merc','audi','bwm' are the top 3 companies thoes sell the most expensive cars.
* another thing we gets to know that is we want to go for budget friendly car than we shoul go with 'hyundai','ford','toyota'..

In [ ]:
dff.boxplot(['mpg'] , by= ['transmission'])

* Manual cars give more mile per galen as comapared to other types.

In [ ]:
dff.boxplot(['mpg'] , by= ['fuelType'])

* cars with hybrid fuelType give  more mpg.
* cars with petrol fuelType give  less mpg.

In [ ]:
dff.boxplot(['mpg'] , by= ['company'] , figsize=(10 ,5))

* from over previous observations we know that toyota company sell the budget range cars. and 
* from this we graphs we get to know that 'toyota' cars gives good mpg as compaired to other companies.
* After toyota there is ford with same measures.



In [ ]:
dff.company.value_counts()

In [ ]:
Carsdata.company.value_counts()

In [ ]:
Carsdata.info()

### **Data preprocessing**

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

In [ ]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
feature_data = Carsdata.drop(columns=['price','model'])      # data with outliers.
target_data = Carsdata.price
###############################################################
feature_data1 = dff.drop(columns=['price','model'])           # data without outliers
target_data1 = dff.price

In [ ]:
cat_data = feature_data.select_dtypes(include=['object'])
print(cat_data.columns)
num_data = feature_data.select_dtypes(include=['int','float'])
print(num_data.columns)

In [ ]:
oe = OrdinalEncoder()
oe.fit(cat_data)
ss = StandardScaler()
ss.fit(num_data)
cat = pd.DataFrame(data = oe.transform(cat_data) , columns=cat_data.columns)
num = pd.DataFrame(data = ss.transform(num_data), columns= num_data.columns)

In [ ]:
cat_pipeline = make_pipeline(OrdinalEncoder())
num_pipeline = make_pipeline(StandardScaler())

In [ ]:
preprocessor = make_column_transformer(
              (cat_pipeline,cat_data.columns),
              (num_pipeline,num_data.columns)
)


In [ ]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)        # data with outliers
##########################################################################
trainX1, testX1, trainY1, testY1 = train_test_split(feature_data1, target_data1)     # data without outliers

### **Linear Regressor**

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
pipeline = make_pipeline(preprocessor, LinearRegression())

* data with outliers.

In [ ]:
pipeline.fit( trainX , trainY)
print("Training Score : ",pipeline.score(trainX, trainY))
print('Testing Score : ',pipeline.score(testX, testY))
print('Root mean square error :',  sqrt(mean_squared_error(testY, pipeline.predict(testX))))

* data without outliers.

In [ ]:
pipeline.fit( trainX1 , trainY1)
print("Training Score : ", pipeline.score( trainX1, trainY1 ))
print('Testing Score : ', pipeline.score( testX1, testY1 ))
print('Root mean square error :', sqrt(mean_squared_error(testY1 , pipeline.predict( testX1 ))))

### **Random Forest Regressor**

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_pipeline = make_pipeline( preprocessor , RandomForestRegressor( n_estimators= 100 ))

* data with outliers.

In [ ]:
rf_pipeline.fit( trainX , trainY)
print("Training Score : ",rf_pipeline.score(trainX, trainY))
print('Testing Score : ',rf_pipeline.score(testX, testY))
print('Root mean square error :',  sqrt(mean_squared_error(testY, rf_pipeline.predict(testX))))

* data without outliers.

In [ ]:
rf_pipeline.fit( trainX1 , trainY1)
print("Training Score : ", rf_pipeline.score( trainX1, trainY1 ))
print('Testing Score : ', rf_pipeline.score( testX1, testY1 ))
print('Root mean square error :', sqrt(mean_squared_error(testY1 , rf_pipeline.predict( testX1 ))))

### **KNeighbors Regressor**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
kn_pipeline = make_pipeline( preprocessor , KNeighborsRegressor(n_neighbors=4))

* data with outliers.

In [ ]:
kn_pipeline.fit( trainX , trainY)
print("Training Score : ",kn_pipeline.score(trainX, trainY))
print('Testing Score : ',kn_pipeline.score(testX, testY))
print('Root mean square error :',  sqrt(mean_squared_error(testY, kn_pipeline.predict(testX))))

* data without outliers.

In [ ]:
kn_pipeline.fit( trainX1 , trainY1)
print("Training Score : ", kn_pipeline.score( trainX1, trainY1 ))
print('Testing Score : ', kn_pipeline.score( testX1, testY1 ))
print('Root mean square error :', sqrt(mean_squared_error(testY1 , kn_pipeline.predict( testX1 ))))

### **Hyper parameter tuning using GridSearchCV**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
gs_pipeline = make_pipeline(preprocessor, RandomForestRegressor(n_estimators=100))

In [ ]:
params = {'randomforestregressor__n_estimators':[100,200,250]}

In [ ]:
gs = GridSearchCV(gs_pipeline, param_grid=params, cv=5, n_jobs=4)

* data with outliers.

In [ ]:
gs.fit( trainX , trainY)
print("Training Score : ",gs.score(trainX, trainY))
print('Testing Score : ',gs.score(testX, testY))
print('Root mean square error :',  sqrt(mean_squared_error(testY, gs.predict(testX))))
print('******************************')
print('Best params :',gs.best_params_)
print('Best Score :', gs.best_score_ )

* data without outliers.

In [ ]:
gs.fit( trainX1 , trainY1)
print("Training Score : ", gs.score( trainX1, trainY1 ))
print('Testing Score : ', gs.score( testX1, testY1 ))
print('Root mean square error :', sqrt(mean_squared_error(testY1 , gs.predict( testX1 ))))
print('******************************')
print('Best params :', gs.best_params_)
print('Best Score :', gs.best_score_ )

* Conclusion 
  * getting the least RMSE of 1909.40 testing model with data without outliers in RendomForestRegress with n_estimetors :250 

### **Results**

In [ ]:
result = pd.DataFrame()
result['actual'] = testY1
result['prediction'] = gs.predict( testX1)
result.sample(10)